# Import libraries

In [3]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
import pandas as pd
import csv

## Functions

In [4]:
def login(email_login, password_login):
    driver.get("https://www.linkedin.com/login/fr")
    #email
    email = driver.find_element_by_name("session_key")
    email.send_keys(email_login)
    #password
    password = driver.find_element_by_name("session_password")
    password.send_keys(password_login)
    #login button
    button = driver.find_element_by_xpath("/html/body/div/main/div[2]/div[1]/form/div[3]/button")
    button.click()

In [5]:
def scroll(timeout):
    scroll_pause_time = timeout

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(scroll_pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # If heights are the same it will exit the function
            break
        last_height = new_height

        # Scroll UP to top to humazine
        driver.execute_script("window.scrollTo(document.body.scrollHeight, 0);")

        # Wait to load page
        time.sleep(scroll_pause_time/2)

In [6]:
def network(timeout, origin):
    driver.get("https://www.linkedin.com/mynetwork/invite-connect/connections/")
    scroll(timeout)

    #Find Names
    list_name = driver.find_elements_by_class_name("mn-connection-card__name")
    NAMES = [name.text for name in list_name]

    #Find Links to profil
    elems = driver.find_elements_by_class_name("mn-connection-card__link")
    LINKS = [elem.get_attribute('href') for elem in elems]

    ORIGIN = [origin] * len(NAMES)

    #Dataset Creation and save
    d = {'Origin' : ORIGIN,
         'Connexion' : NAMES,
         'Links' : LINKS}
    df = pd.DataFrame(data = d)
    df.to_csv('out.csv', index=True)

In [7]:
def get_network(name):
    df = pd.read_csv('out.csv')
    #go to given profil
    link = df[df["Connexion"] == name].Links
    driver.get(link.values[0])

    origin = name
    NAMES = []

    # network button
    try :
        button = driver.find_element_by_xpath("/html/body/div[7]/div[3]/div/div/div/div/div[2]/div/div/main/div/section/div[2]/ul/li/a")
        button.click()
    except:
        print("no network button")
        return NAMES 
    

    #get number of pages
    scroll(4)
    # If number of pages is higher than 10
    try :
        pages = driver.find_element_by_xpath("/html/body/div[7]/div[3]/div/div[1]/div/div[1]/main/div/div/div[4]/div/div/ul/li[10]/button")
        pages = int(pages.text)
    except:
        pageButton = driver.find_elements_by_class_name("artdeco-pagination__indicator--number")
        pageButton = [page.text for page in pageButton]
        pageButton.pop(-2)# remove '...'
        pageButton = [int(page) for page in pageButton]
        pages = max(pageButton)
    
    print("number of pages detected = " + str(pages))
        

    # Find Names
    for i in range(0, pages):
        list_name = driver.find_elements_by_class_name("entity-result__title-text")
        names = [name.text for name in list_name]
        names = [name.split("\n",1)[0] for name in names]
        NAMES.append(names)

        # next page button
        nextButton = driver.find_element_by_class_name("artdeco-pagination__button--next")
        nextButton.click()

        scroll(2) #mandatory

    return NAMES

In [8]:
def flatten_and_save(NameList, name):
    #flatten
    flatten_NAMES = []
    for i in range (0, len(NameList) -1 ):
        for j in range(0,len(NameList[0])):
            flatten_NAMES.append(NameList[i][j])
    
    #flatten last row which might be different         
    for i in range(0,len(NameList[-1])):
        flatten_NAMES.append(NameList[-1][i])
            
    #save
    ORIGIN = [name] * len(flatten_NAMES)

    #Dataset Creation and save
    d = {'Origin' : ORIGIN,
         'Connexion' : flatten_NAMES}
    
    df1 = pd.read_csv('out_final.csv')
    df2 = pd.DataFrame(data = d)
    df3 = pd.concat([df1, df2],  join = 'inner')
    df3 = df3.drop_duplicates()
    df3.to_csv('out_final.csv', index=True)

In [9]:
def export():
    df = pd.read_csv('out_final.csv')
    df = df.rename(columns={"Origin": "source", "Connexion": "target"})
    data = df[["source", "target"]].values.tolist()
    header = df[["source", "target"]].columns.values.tolist()

    with open('gephi.csv', 'w', encoding='UTF8') as f:
        writer = csv.writer(f)

        # write the header
        writer.writerow(header)

        # write the data
        writer.writerows(data)

## Lauch and connect

In [35]:
PATH_DRIVER = r"/Users/FredericGodest/PycharmProjects/linkedinscrap/chromedriver"
PATH_MAIN = "https://www.linkedin.com/mynetwork/invite-connect/connections/"
driver = webdriver.Chrome(PATH_DRIVER)

print("Your Email ?")
email = input()
print("Your Linkedin password ?")
mdp = input()
print("Your name ?")
origin = input()


login(email, mdp)

## Get your network

In [ ]:
network(5, origin)

## Get network of someone

In [ ]:
print("Name of the person scanned ? ")
name = input()
NAMES = get_network(name)
flatten_and_save(NAMES, name)
export()